# Retreiving and Preparing Stock's Balance Sheet Information
This section of the code will concentrate on extracting stock's balance sheet information from IEX Cloud API. At the end, this section of the code will provide a complete and formatted balance sheet dataset.  

## Importing neccessary modules

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import xlsxwriter

## Setting up base URL

Fetching the fundamentals and finacial data about the stock from ther IEX Cloud endpoints.

In [3]:
from api_secret import IEX_CLOUD_API_TOKEN

symbol = 'FTNT'# input("Enther the ticker here: ")

# API endpoints to collect stock data
balance_sheet_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/balance-sheet?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
financials_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/financials?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
fundamentals_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/fundamentals?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'

balance_sheet_endpoint_data = requests.get(balance_sheet_endpoint).json()
financials_endpoint_data = requests.get(financials_endpoint).json()
fundamentals_endpoint_data = requests.get(fundamentals_endpoint).json()


## Extracting the data
This part will concentrate on extracting the data that will form the final balance sheet for the stock.

### Forming the Dataset columns
The columns will be comprised of the Pandas DatetimeIndex. It will be created based on the quarters (reportDate) of the reported stock data.


Finding quarters.

In [4]:
reportDate_list = []
quarter_list = []

# Converting string date into the date object
for stock_data in balance_sheet_endpoint_data['balancesheet']:
    converted_date = datetime.strptime(stock_data['reportDate'], '%Y-%m-%d').date()
    reportDate_list.append(converted_date)


# function to determine to which quarter the reported stock data
# is referred to
def quarter_finder(dates_list):
    global quarter_list

    for d in dates_list:
        q1 = date(d.year, 3, 31)
        q2 = date(d.year, 6, 30)
        q3 = date(d.year, 9, 30)
        q4 = date(d.year, 12, 31)

        if d < q1:
            q = date(d.year - relativedelta(year=1).year, 12, 31)
        if d > q1 and d < q2:
            q = date(d.year, 3, 30)
        if d > q2 and d < q3:
            q = date(d.year, 6, 30)
        if d > q3 and d < q4:
            q = date(d.year, 9, 30)
        
        quarter_list.append(q)


quarter_finder(reportDate_list)

The above created Datetime-like array will serve as an input to Pandas' DatetimeIndex objet (columns of the future dataset)

In [5]:
columns = pd.DatetimeIndex(data=quarter_list)

The next step is the creation of the actual DataFrame.

In [6]:
index = [
    # ---- Assets ----
    'Total Current Assets',
    'Total Current Assets (Growth)'
    'Cash',
    'Cash (Growth)',
    'Short Term Investments',
    'Short Term Investments (Growth)',
    'Accounts Receivable',
    'Accounts Receivable (Growth)',
    'Inventories',
    'Inventories (Growth)',
    'PP&E',
    'PP&E (Growth)',
    'Accumulated Depreciation',
    'Goodwill',
    'Intangible Assets',
    'Long Term Investments',
    'Long Term Investments (Growth)',
    # ---- Liabilities ----
    'Total Liabilities',
    'Total Liabilities (Growth)',
    'Current Liabilities',
    'Current Liabilities (Growth)',
    'Accounts Payable',
    'Accounts Payable (Growth)',
    'Income Tax Payable',
    'Current Deferred Revenue',
    'Current Deferred Revenue (Growth)',
    'Total Non Current Liabilities',
    'Total Non Current Liabilities (Growth)',
    'Long Term Debt',
    'Long Term Debt (Growth)',
    'Deferred Taxes',
    # ---- Shareholders Equity ----
    'Shareholders Equity',
    'Shareholders Equity (Growth)',
    'Retained Earnings',
    'Retained Earnings (Growth)'
]

balance_sheet = pd.DataFrame(columns=columns, index=index, data=None)

### Filling the dataframe with data returned by API endpoints.

The first API endpoint to get the stock data from is the 'balance-sheet' endpoint.

In [7]:
balance_sheet_dict = {
    'Total Current Assets': [],
    'Cash': [],
    'Short Term Investments': [],
    'Accounts Receivable': [],
    'Inventories': [],
    'PP&E': [],
    'Goodwill': [],
    'Intangible Assets': [],
    'Long Term Investments': [],
    'Total Liabilities': [],
    'Current Liabilities': [],
    'Accounts Payable': [],
    'Long Term Debt': [],
    'Shareholders Equity': [],
    'Retained Earnings': []
}


# Filling the balance_sheet_dict dictionary with data collected
# from the API endpoint
for quarter_data in balance_sheet_endpoint_data['balancesheet']:
    balance_sheet_dict['Total Current Assets'].append(quarter_data['currentAssets'])
    balance_sheet_dict['Cash'].append(quarter_data['currentCash'])
    balance_sheet_dict['Short Term Investments'].append(quarter_data['shortTermInvestments'])
    balance_sheet_dict['Accounts Receivable'].append(quarter_data['receivables'])
    balance_sheet_dict['Inventories'].append(quarter_data['inventory'])
    balance_sheet_dict['PP&E'].append(quarter_data['propertyPlantEquipment'])
    balance_sheet_dict['Goodwill'].append(quarter_data['goodwill'])
    balance_sheet_dict['Intangible Assets'].append(quarter_data['intangibleAssets'])
    balance_sheet_dict['Long Term Investments'].append(quarter_data['longTermInvestments'])
    balance_sheet_dict['Total Liabilities'].append(quarter_data['totalLiabilities'])
    balance_sheet_dict['Current Liabilities'].append(quarter_data['longTermInvestments'])
    balance_sheet_dict['Accounts Payable'].append(quarter_data['totalCurrentLiabilities'])
    balance_sheet_dict['Long Term Debt'].append(quarter_data['longTermDebt'])
    balance_sheet_dict['Shareholders Equity'].append(quarter_data['shareholderEquity'])
    balance_sheet_dict['Retained Earnings'].append(quarter_data['retainedEarnings'])


# Putting the data into the DataFrame
for stock_attribute in balance_sheet_dict:
    for row in balance_sheet.index:
        if row == stock_attribute:
            balance_sheet.loc[row] = balance_sheet_dict[stock_attribute]

Moving on to the 'financials' endpoint.

In [8]:
financials_dict = {
    'Accumulated Depreciation': []
}


# Filling the financials_dict dictionary with data collected
# from the API endpoint
for quarter_data in financials_endpoint_data['financials']:
    financials_dict['Accumulated Depreciation'].append(quarter_data['depreciation'])



# Putting the data into the DataFrame
for stock_attribute in financials_dict:
    for row in balance_sheet.index:
        if row == stock_attribute:
            balance_sheet.loc[row] = financials_dict[stock_attribute]

FInally, the take on 'fundamentals' API endpoint.

In [9]:
fundamentals_dict = {
    'Income Tax Payable': [],
    'Current Deferred Revenue': [],
    'Deferred Taxes': [],
    'Total Non Current Liabilities': [],
}


# Filling the fundamentals_dict dictionary with data collected
# from the API endpoint
for quarter_data in fundamentals_endpoint_data['fundamentals']:
    fundamentals_dict['Income Tax Payable'].append(quarter_data['incomeTaxCurrent'])
    fundamentals_dict['Current Deferred Revenue'].append(quarter_data['nibclRevenueDeferred'])
    fundamentals_dict['Deferred Taxes'].append(quarter_data['incomeTaxDeferred'])
    fundamentals_dict['Total Non Current Liabilities'].append(quarter_data['liabilitiesNonCurrentAndInterestMinorityTotal'])


# Putting the data into the DataFrame
for stock_attribute in fundamentals_dict:
    for row in balance_sheet.index:
        if row == stock_attribute:
            balance_sheet.loc[row] = fundamentals_dict[stock_attribute]

## Calculating the Growth


Below section deals with the 'Growth' columns to give an investor an advantage of company's success metric.

In [10]:
# Calculating the growth rate based on the stock metrics' 
for df_index, row_name in enumerate(balance_sheet.index):
    if 'Growth' in row_name:
        for row_index, value in enumerate(balance_sheet.iloc[df_index - 1]):
            try:
                growth_rate = (value / balance_sheet.iloc[df_index - 1, row_index + 1]) * 100
                balance_sheet.iloc[df_index, row_index] = (growth_rate - 100) / 100
            except ZeroDivisionError:
                growth_rate = np.nan
            except IndexError:
                break

Handling NaN values.

In [11]:
balance_sheet.fillna(value='', inplace=True)
balance_sheet

,2022-09-30,2022-06-30,2022-03-30,2021-12-31,2021-09-30,2021-06-30,2021-03-30,2020-12-31,2020-09-30,2020-06-30,2020-03-30,2019-12-31
Total Current Assets,3.006805e+09,3.011636e+09,3.297436e+09,3.706277e+09,4.033323e+09,4.008114e+09,3812656152,2774963069,2401884458,2300545335,2127589371,2899688120
Total Current Assets (Growth)Cash,-1.603821e-03,-8.667364e-02,-1.103104e-01,-8.108577e-02,6.289423e-03,5.126549e-02,0.373948,0.155327,0.04405,0.081292,-0.26627,
Cash (Growth),-9.814959e-01,-2.142753e-01,3.604168e-01,-1.389240e+01,-8.773166e-01,-8.629076e-01,1.407484,2.526159,-0.458126,-1.3053,,
Short Term Investments,7.430814e+07,8.364628e+07,9.304072e+07,6.575747e+07,6.020946e+07,6.112565e+07,59213308,44943575,49106099,59005875,56864737,42256434
Short Term Investments (Growth),-1.116383e-01,-1.009713e-01,4.149072e-01,9.214520e-02,-1.498865e-02,3.229583e-02,0.317503,-0.084766,-0.167776,0.037653,0.345706,
Accounts Receivable,9.957658e+08,9.446597e+08,7.917912e+08,8.078497e+08,6.140025e+08,6.033953e+08,655823795,737539404,561719979,504931995,475442308,557914660
Accounts Receivable (Growth),5.410010e-02,1.930666e-01,-1.987810e-02,3.157107e-01,1.757922e-02,-7.994293e-02,-0.110795,0.313002,0.112467,0.062026,-0.147823,
Inventories,2.259741e+08,1.988071e+08,1.908692e+08,1.835305e+08,1.836690e+08,1.498969e+08,152109939,146211208,137074768,129887885,109384425,119904661
Inventories (Growth),1.366500e-01,4.158840e-02,3.998598e-02,-7.538127e-04,2.253017e-01,-1.454873e-02,0.040344,0.066653,0.055331,0.187444,-0.087738,
PP&E,9.119651e+08,8.467639e+08,8.196048e+08,6.990024e+08,5.801127e+08,5.287052e+08,506670897,464746268,423474459,411126273,384994831,359146389
